In [1]:
import pandas as pd

In [2]:
df_metacoll = pd.read_csv(r"D:\datasets\FSD50K\FSD50K.metadata\collection\collection_dev.csv")

df_metacoll

,fname,labels,mids
0,64760,Electric_guitar,/m/02sgy
1,16399,Electric_guitar,/m/02sgy
2,16401,Electric_guitar,/m/02sgy
3,16402,Electric_guitar,/m/02sgy
4,16404,Electric_guitar,/m/02sgy
...,...,...,...
40961,102863,Goose,/m/0dbvp
40962,389607,Goose,/m/0dbvp
40963,90091,Duck,/m/09ddx
40964,244718,Duck,/m/09ddx


In [3]:
df_metacoll.groupby("mids").count()

,fname,labels
mids,,
/g/122z_qxw,30,30
/m/011k_j,55,55
"/m/011k_j,/m/0l14md",1,1
/m/012f08,73,73
"/m/012f08,/m/01bjv",5,5
...,...,...
"/t/dd00135,/m/07p6fty",1,1
"/t/dd00135,/m/07sr1lc",5,5
"/t/dd00135,/t/dd00013",1,1


In [4]:
import json

with open(r"C:\Users\memis\Documents\mct\thesis\code\analyze_datasets\ontology.json") as f:
    mids = json.load(f)
dict_mids = {d["id"]: d for d in mids}

dict_mids

{'/m/0dgw9r': {'id': '/m/0dgw9r',
  'name': 'Human sounds',
  'description': 'Sounds produced by the human body through the actions of the individual.',
  'citation_uri': '',
  'positive_examples': [],
  'child_ids': ['/m/09l8g',
   '/m/01w250',
   '/m/09hlz4',
   '/m/0bpl036',
   '/m/0160x5',
   '/m/0k65p',
   '/m/01jg02',
   '/m/04xp5v',
   '/t/dd00012'],
  'restrictions': ['abstract']},
 '/m/09l8g': {'id': '/m/09l8g',
  'name': 'Human voice',
  'description': 'The human voice consists of sound made by a human being using the vocal folds for talking, singing, laughing, crying, screaming, etc. The human voice is specifically a part of human sound production in which the vocal folds are the primary sound source.',
  'citation_uri': 'http://en.wikipedia.org/wiki/Human_voice',
  'positive_examples': [],
  'child_ids': ['/m/09x0r',
   '/m/07p6fty',
   '/m/03qc9zr',
   '/m/02rtxlg',
   '/m/01j3sz',
   '/m/0463cq4',
   '/m/07qw_06',
   '/m/07plz5l',
   '/m/015lz1',
   '/m/02fxyj',
   '/m/07

In [34]:
len(set([k for k, v in dict_mids.items()]))

632

In [7]:
first_level = set(dict_mids.keys())
for ki in dict_mids.keys():
    for kj, vd in dict_mids.items():
        if ki in vd["child_ids"]:
            first_level.remove(ki)
            break

first_level

{'/m/04rlf',
 '/m/059j3w',
 '/m/0dgw9r',
 '/m/0jbk',
 '/t/dd00041',
 '/t/dd00098',
 '/t/dd00123'}

In [8]:
visited = []
def add_parents(k, parents=[]):
    global visited
    visited.append(k)
    if "parents" in dict_mids[k].keys():
        dict_mids[k]["parents"].extend(parents)
    else:
        dict_mids[k]["parents"] = parents
    for k_child in dict_mids[k]["child_ids"]:
        add_parents(k_child, parents+[k])

for ch in first_level:
    add_parents(ch)

In [9]:
len(visited), len(set(visited))

(690, 632)

In [149]:
#outdated
"""
counter = 0
for ki in dict_mids.keys():
    for kj, v_dict in dict_mids.items():
        if ki in v_dict["child_ids"]:
            if "parents" in dict_mids[ki].keys():
                dict_mids[ki]["parents"].append(kj)
            else:
                dict_mids[ki]["parents"] = [kj]
    counter += 1
    print(counter)
"""
pass

In [10]:
d_lens = {}
for k, v in dict_mids.items():
    if k in d_lens.keys():
        input("WTF")
    d_lens[k] = len(v["parents"]) if "parents" in v.keys() else None

In [11]:
l_lens = [v for k, v in d_lens.items() if v is not None]

max(l_lens)

12

In [12]:
max_lens = [(k, v) for k, v in d_lens.items() if v == 2]

len(max_lens)

277

In [13]:
"/m/0dgw9r"
"/t/dd00098"
"/m/0jbk"
"/t/dd00041"
...

Ellipsis

In [14]:
first_level = set(dict_mids.keys())
for ki in dict_mids.keys():
    for kj, vd in dict_mids.items():
        if ki in vd["child_ids"]:
            first_level.remove(ki)
            break

first_level

{'/m/04rlf',
 '/m/059j3w',
 '/m/0dgw9r',
 '/m/0jbk',
 '/t/dd00041',
 '/t/dd00098',
 '/t/dd00123'}

In [ ]:
second_level = []
for mid in first_level:
    second_level.extend(dict_mids[mid]["child_ids"])

second_level

In [ ]:
third_level = []
for mid in second_level:
    third_level.extend(dict_mids[mid]["child_ids"])

third_level

In [40]:
for k, v in dict_mids.items():
    if not "parents" in v.keys() or len(v["parents"]) == 0:
        assert k in first_level, str(k)
    elif "parents" in v.keys():
        fl = set(v["parents"]).intersection(first_level)
        sl = set(v["parents"]).intersection(second_level)
        assert len(fl) > 0, (str(k), v["parents"])
        assert len(sl) > 0 or k in second_level, (str(k), v["parents"])


In [ ]:
for k, v in dict_mids.items():
    if len(v["parents"]) != len(set(v["parents"])):
        print(k, v["parents"])
        input()

In [95]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges to the graph
exists = []
for mid, data in dict_mids.items():
    # Add the node
    if mid in exists:
        input(f"altready exists {mid}")
    else:
        G.add_node(mid)
        exists.append(mid)
    # # Add edges from parents to current node
    # for parent in data["parents"]:
    #     G.add_edge(parent, mid)
    # Add edges from current node to children
    for child in data["child_ids"]:
        G.add_edge(mid, child)

len(set(G.nodes())), len(G.nodes())

(632, 632)

In [97]:
layers = {}
layers[0] = list(first_level)
level = list(first_level)
level_id = 0
visited = []
visited.extend(first_level)
while True:
    level_id += 1

    next_level = []
    for mid in level:
        for child in dict_mids[mid]["child_ids"]:
            if child not in visited:
                next_level.append(child)
                visited.append(child)
    if len(next_level) == 0:
        break

    layers[level_id] = next_level
    level = next_level

len(set(layers)), len(visited), layers.keys()

(6, 632, dict_keys([0, 1, 2, 3, 4, 5]))

In [98]:
for node in G.nodes():
    assert node in visited, "1"

for node in visited:
    assert node in G.nodes(), "2"

In [99]:
len(G), sum(len(nodes) for nodes in layers.values())

(632, 632)

In [105]:
!pip install PyQt5

     ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
     - -------------------------------------- 0.2/6.9 MB 7.6 MB/s eta 0:00:01
     ---- ----------------------------------- 0.7/6.9 MB 8.9 MB/s eta 0:00:01
     ----- ---------------------------------- 1.0/6.9 MB 8.1 MB/s eta 0:00:01
     -------- ------------------------------- 1.5/6.9 MB 8.6 MB/s eta 0:00:01
     ---------- ----------------------------- 1.8/6.9 MB 8.1 MB/s eta 0:00:01
     ------------- -------------------------- 2.3/6.9 MB 8.5 MB/s eta 0:00:01
     --------------- ------------------------ 2.6/6.9 MB 8.3 MB/s eta 0:00:01
     ----------------- ---------------------- 3.0/6.9 MB 8.4 MB/s eta 0:00:01
     ------------------- -------------------- 3.4/6.9 MB 8.3 MB/s eta 0:00:01
     ---------------------- ----------------- 3.8/6.9 MB 8.4 MB/s eta 0:00:01
     ------------------------ --------------- 4.2/6.9 MB 8.4 MB/s eta 0:00:01
     -------------------------- ------------- 4.6/6.9 MB 8.4 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [107]:
%matplotlib qt

In [108]:
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
# Use graphviz_layout for hierarchical layout
pos = nx.multipartite_layout(G, subset_key=layers, align='horizontal')

# Draw the graph with hierarchical layout
nx.draw(G, pos, with_labels=True, node_size=700, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=20)
plt.title("Top-Down Tree Visualization of Ontology")
plt.show()

In [86]:
first_level

{'/m/04rlf',
 '/m/059j3w',
 '/m/0dgw9r',
 '/m/0jbk',
 '/t/dd00041',
 '/t/dd00098',
 '/t/dd00123'}

In [94]:
len(second_level), len(set(second_level))

(43, 43)

In [96]:
len(third_level), len(set(third_level))

(313, 306)

In [51]:
dict_mids["/m/0dgw9r"]

{'id': '/m/0dgw9r',
 'name': 'Human sounds',
 'description': 'Sounds produced by the human body through the actions of the individual.',
 'citation_uri': '',
 'positive_examples': [],
 'child_ids': ['/m/09l8g',
  '/m/01w250',
  '/m/09hlz4',
  '/m/0bpl036',
  '/m/0160x5',
  '/m/0k65p',
  '/m/01jg02',
  '/m/04xp5v',
  '/t/dd00012'],
 'restrictions': ['abstract']}

In [52]:
dict_mids["/m/09l8g"]

{'id': '/m/09l8g',
 'name': 'Human voice',
 'description': 'The human voice consists of sound made by a human being using the vocal folds for talking, singing, laughing, crying, screaming, etc. The human voice is specifically a part of human sound production in which the vocal folds are the primary sound source.',
 'citation_uri': 'http://en.wikipedia.org/wiki/Human_voice',
 'positive_examples': [],
 'child_ids': ['/m/09x0r',
  '/m/07p6fty',
  '/m/03qc9zr',
  '/m/02rtxlg',
  '/m/01j3sz',
  '/m/0463cq4',
  '/m/07qw_06',
  '/m/07plz5l',
  '/m/015lz1',
  '/m/02fxyj',
  '/m/07s2xch',
  '/m/07r4k75',
  '/m/01j423'],
 'restrictions': ['abstract']}

In [53]:
dict_mids["/m/09x0r"]

{'id': '/m/09x0r',
 'name': 'Speech',
 'description': 'Speech is the vocalized form of human communication, created out of the phonetic combination of a limited set of vowel and consonant speech sound units.',
 'citation_uri': 'http://en.wikipedia.org/wiki/Speech',
 'positive_examples': ['youtu.be/8uI9H5jGRV8?start=30&end=40',
  'youtu.be/cz8QIJHjZh4?start=30&end=40',
  'youtu.be/91FIbeKyIhw?start=30&end=40',
  'youtu.be/V3HdocEv8gw?start=220&end=230',
  'youtu.be/a8igK2oWlVY?start=30&end=40',
  'youtu.be/2hvTtM7VdJg?start=30&end=40',
  'youtu.be/S7ytQItOrkk?start=30&end=40',
  'youtu.be/Zyl2Q3GepPA?start=30&end=40',
  'youtu.be/CU1qRm_Bcps?start=30&end=40'],
 'child_ids': ['/m/05zppz',
  '/m/02zsn',
  '/m/0ytgt',
  '/m/01h8n0',
  '/m/02qldy',
  '/m/0261r1',
  '/m/0brhx'],
 'restrictions': []}

In [38]:
import json

with open(r"D:\datasets\FSD50K\FSD50K.metadata\dev_clips_info_FSD50K.json") as f:
    clips = json.load(f)

clips

{'425873': {'title': 'water baltic sea foley water run in fast splashes heavy mono.wav',
  'description': 'This sound effect was recorded with high quality sound equipment and comes from a sound library called Baltic Sea which is pack of 56 high quality audio files with a total length of 153 minutes. In this library you\'ll find various ambients recorded on the shores of the Baltic Sea. You can use this sound for free, and if you need a whole package of ambients you can support my work and get it for 40$ here: <a href="http://soundholder.com/product/sound-library-baltic-sea/" rel="nofollow">http://soundholder.com/product/sound-library-baltic-sea/</a>\n\nAlso here\'s a 10% discount code for all freesound users, just type : freesound in discount window.\n\nEnjoy!',
  'tags': ['splashes',
   'foley',
   'soundscape',
   'baltic',
   'ambience',
   'ambient',
   'sea',
   'location',
   'atmo',
   'water',
   'wave',
   'whoosh',
   'waves',
   'swell',
   'splash',
   'run',
   'noise',
 

In [20]:
clips['64760']

{'title': 'guitarras_63.wav',
 'description': 'electric guitar',
 'tags': ['electric', 'guitar'],
 'license': 'http://creativecommons.org/licenses/sampling+/1.0/',
 'uploader': 'casualsamples'}